# MIGRATING UPDATED DATA TO THE DATABASE:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector
import csv
import json
from prettytable import PrettyTable
import random
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Loading the new data:

In [3]:
csv_file = "data/new_data.csv"
df = pd.read_csv(csv_file)

In [4]:
df.shape

(50635, 31)

This new data has 50635 rows that contain updated information about the crimes in chicago

In [21]:
with open('config_db.json') as config_json:
    config = json.load(config_json)

conx = mysql.connector.connect(**config) 

In [6]:
mycursor = conx.cursor()
query = "DESCRIBE crimes"
mycursor.execute(query)

description_table = mycursor.fetchall()

mycursor.close()

# we have seen the creation of the table 
desc_table=pd.DataFrame(description_table, columns=['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'])
print(desc_table)

mycursor.close()

            Field             Type Null  Key Default Extra
0              id           b'int'   NO  PRI    None      
1            date          b'date'  YES         None      
2            time          b'time'  YES         None      
3           block  b'varchar(150)'  YES         None      
4            iucr   b'varchar(10)'  YES  MUL    None      
5   location_desc  b'varchar(150)'  YES         None      
6          arrest    b'tinyint(1)'  YES         None      
7        district           b'int'  YES         None      
8            year           b'int'  YES         None      
9      updated_on      b'datetime'  YES         None      
10       location         b'point'  YES         None      


False

#### We have to make sure that the new data fits into this table crimes, now let's do some transformations to it:

In [7]:
df.tail(10)

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,location,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu,:@computed_region_rpca_8um6,:@computed_region_d9mm_jgwp,:@computed_region_d3ds_rm58,:@computed_region_8hcu_yrd4
50625,13141337,JG341811,2023-07-15T08:20:00.000,076XX S COLFAX AVE,0460,BATTERY,SIMPLE,APARTMENT,True,True,...,"{'latitude': '41.756650207', 'longitude': '-87...",43.0,22538.0,39.0,245.0,37.0,24.0,19.0,225.0,5.0
50626,13141238,JG341741,2023-07-15T08:19:00.000,075XX S SANGAMON ST,0454,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",STREET,True,False,...,"{'latitude': '41.757118414', 'longitude': '-87...",17.0,21554.0,70.0,572.0,31.0,59.0,20.0,229.0,17.0
50627,13141228,JG341722,2023-07-15T08:15:00.000,056XX W NORTH AVE,041A,BATTERY,AGGRAVATED - HANDGUN,STREET,False,False,...,"{'latitude': '41.909375517', 'longitude': '-87...",52.0,22615.0,26.0,597.0,7.0,5.0,6.0,154.0,29.0
50628,13153634,JG356482,2023-07-15T08:00:00.000,028XX W 24TH BLVD,0915,MOTOR VEHICLE THEFT,"TRUCK, BUS, MOTOR HOME",SCHOOL - PUBLIC GROUNDS,False,False,...,"{'latitude': '41.847867226', 'longitude': '-87...",29.0,21569.0,32.0,262.0,1.0,57.0,21.0,160.0,24.0
50629,13142620,JG343529,2023-07-15T08:00:00.000,035XX W 63RD ST,0820,THEFT,$500 AND UNDER,CONVENIENCE STORE,False,False,...,"{'latitude': '41.778844906', 'longitude': '-87...",44.0,21867.0,64.0,344.0,6.0,58.0,13.0,276.0,14.0
50630,13141484,JG342175,2023-07-15T08:00:00.000,039XX W FLOURNOY ST,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,"{'latitude': '41.872559142', 'longitude': '-87...",36.0,21572.0,27.0,702.0,14.0,30.0,16.0,142.0,24.0
50631,13141591,JG342310,2023-07-15T08:00:00.000,067XX S CRANDON AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,False,False,...,"{'latitude': '41.772711564', 'longitude': '-87...",32.0,22538.0,39.0,134.0,33.0,24.0,18.0,262.0,3.0
50632,13141246,JG341856,2023-07-15T08:00:00.000,071XX S PAXTON AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,"{'latitude': '41.765281136', 'longitude': '-87...",32.0,22538.0,39.0,453.0,33.0,24.0,18.0,89.0,3.0
50633,13145984,JG347492,2023-07-15T08:00:00.000,052XX S KEATING AVE,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER (SPECIFY),False,True,...,"{'latitude': '41.79791797', 'longitude': '-87....",35.0,22248.0,53.0,173.0,8.0,56.0,13.0,105.0,23.0
50634,13164141,JG369029,2023-07-15T08:00:00.000,075XX N WASHTENAW AVE,1120,DECEPTIVE PRACTICE,FORGERY,SIDEWALK,False,False,...,"{'latitude': '42.018832815', 'longitude': '-87...",42.0,22528.0,20.0,359.0,27.0,12.0,11.0,35.0,50.0


In [8]:
df = df.iloc[:, :-10]
df.head(10)

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude
0,13218359,JG433539,2023-09-21T00:00:00.000,114XX S UNION AVE,0820,THEFT,$500 AND UNDER,STREET,False,False,...,22,21.0,49,06,1173696.0,1828978.0,2023,2023-09-28T15:42:39.000,41.686124,-87.639761
1,13220574,JG435752,2023-09-21T00:00:00.000,070XX N Sheridan Rd,0820,THEFT,$500 AND UNDER,STREET,False,False,...,24,49.0,1,06,1166538.0,1947205.0,2023,2023-09-28T15:42:39.000,42.010705,-87.662584
2,13222674,JG434826,2023-09-21T00:00:00.000,007XX N LA SALLE DR,0460,BATTERY,SIMPLE,COMMERCIAL / BUSINESS OFFICE,False,False,...,18,42.0,8,08B,1175002.0,1905476.0,2023,2023-09-28T15:42:39.000,41.896013,-87.632698
3,13219537,JG435105,2023-09-21T00:00:00.000,034XX W CULLOM AVE,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,...,17,33.0,16,06,1152712.0,1928368.0,2023,2023-09-28T15:42:39.000,41.959301,-87.713957
4,13217762,JG432890,2023-09-21T00:00:00.000,0000X S COLUMBUS DR,0920,MOTOR VEHICLE THEFT,ATTEMPT - AUTOMOBILE,STREET,False,False,...,1,42.0,32,07,1178293.0,1900315.0,2023,2023-09-28T15:42:39.000,41.881777,-87.620768
5,13217613,JG432727,2023-09-21T00:00:00.000,059XX N KENMORE AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,24,48.0,77,07,1168163.0,1939796.0,2023,2023-09-28T15:42:39.000,41.990339,-87.656821
6,13217536,JG432562,2023-09-21T00:00:00.000,008XX W 69TH ST,031B,ROBBERY,ARMED - OTHER FIREARM,STREET,False,False,...,7,16.0,68,03,1171685.0,1859095.0,2023,2023-09-28T15:42:39.000,41.768813,-87.646243
7,13218479,JG433800,2023-09-21T00:00:00.000,105XX S INDIANA AVE,4387,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,RESIDENCE,False,True,...,5,9.0,49,26,1179445.0,1835176.0,2023,2023-09-28T15:42:39.000,41.703003,-87.618527
8,13217798,JG432794,2023-09-21T00:00:00.000,016XX W HARRISON ST,0820,THEFT,$500 AND UNDER,STREET,False,False,...,12,27.0,28,06,1165781.0,1897452.0,2023,2023-09-28T15:42:39.000,41.874196,-87.666793
9,13218255,JG433566,2023-09-21T00:00:00.000,065XX S ELLIS AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,APARTMENT,False,False,...,3,20.0,42,14,1184012.0,1861912.0,2023,2023-09-28T15:42:39.000,41.776264,-87.600970


#### We removed the last 9 columns that were information of the API, but not relevant to the Crimes dataset. Also the column location was deleted we have to transform the columns latitude and longitude to create the new location column with the POINT format of MySQL:

In [9]:

not_null_mask = ~df['latitude'].isnull() & ~df['longitude'].isnull()

# Apply the transformation only for non-null values
df.loc[not_null_mask, 'location'] = df[not_null_mask].apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)
df

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,13218359,JG433539,2023-09-21T00:00:00.000,114XX S UNION AVE,0820,THEFT,$500 AND UNDER,STREET,False,False,...,21.0,49,06,1173696.0,1828978.0,2023,2023-09-28T15:42:39.000,41.686124,-87.639761,POINT(41.686123758 -87.639760794)
1,13220574,JG435752,2023-09-21T00:00:00.000,070XX N Sheridan Rd,0820,THEFT,$500 AND UNDER,STREET,False,False,...,49.0,1,06,1166538.0,1947205.0,2023,2023-09-28T15:42:39.000,42.010705,-87.662584,POINT(42.010704854 -87.662584365)
2,13222674,JG434826,2023-09-21T00:00:00.000,007XX N LA SALLE DR,0460,BATTERY,SIMPLE,COMMERCIAL / BUSINESS OFFICE,False,False,...,42.0,8,08B,1175002.0,1905476.0,2023,2023-09-28T15:42:39.000,41.896013,-87.632698,POINT(41.896013242 -87.632697553)
3,13219537,JG435105,2023-09-21T00:00:00.000,034XX W CULLOM AVE,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,...,33.0,16,06,1152712.0,1928368.0,2023,2023-09-28T15:42:39.000,41.959301,-87.713957,POINT(41.959301089 -87.713956891)
4,13217762,JG432890,2023-09-21T00:00:00.000,0000X S COLUMBUS DR,0920,MOTOR VEHICLE THEFT,ATTEMPT - AUTOMOBILE,STREET,False,False,...,42.0,32,07,1178293.0,1900315.0,2023,2023-09-28T15:42:39.000,41.881777,-87.620768,POINT(41.881776822 -87.620767948)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50630,13141484,JG342175,2023-07-15T08:00:00.000,039XX W FLOURNOY ST,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,24.0,26,05,1150263.0,1896740.0,2023,2023-08-19T15:40:26.000,41.872559,-87.723787,POINT(41.872559142 -87.723786945)
50631,13141591,JG342310,2023-07-15T08:00:00.000,067XX S CRANDON AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,False,False,...,5.0,43,08A,1192484.0,1860691.0,2023,2023-08-19T15:40:26.000,41.772712,-87.569953,POINT(41.772711564 -87.569952861)
50632,13141246,JG341856,2023-07-15T08:00:00.000,071XX S PAXTON AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,5.0,43,07,1192124.0,1857980.0,2023,2023-08-19T15:40:26.000,41.765281,-87.571361,POINT(41.765281136 -87.57136054)
50633,13145984,JG347492,2023-07-15T08:00:00.000,052XX S KEATING AVE,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER (SPECIFY),False,True,...,23.0,56,26,1145578.0,1869507.0,2023,2023-08-19T15:40:26.000,41.797918,-87.741677,POINT(41.79791797 -87.741676545)


In [10]:
df.isna().sum()

id                        0
case_number               0
date                      0
block                     0
iucr                      0
primary_type              0
description               0
location_description    197
arrest                    0
domestic                  0
beat                      0
district                  0
ward                      1
community_area            0
fbi_code                  0
x_coordinate            338
y_coordinate            338
year                      0
updated_on                0
latitude                338
longitude               338
location                338
dtype: int64

#### Now we apply the transformations to the column Date to divite in date and time, replace nulls with 0 and changing the data type:

In [11]:
df['time'] = df['date'].str.split('T').str[1]
df['date'] = df['date'].str.split('T').str[0]

# replacing nulls
df = df.fillna({'location_description': 'null', 'ward': 0, 'x_coordinate': 0,'y_coordinate': 0})

# cambiar el tipo de dato de las columnas

cols = ['district', 'arrest', 'domestic', 'ward', 'community_area', 'x_coordinate', 'y_coordinate']
df[cols] = df[cols].astype(int)

move_time = df.pop('time')
df.insert(3,'time', move_time)

df

,id,case_number,date,time,block,iucr,primary_type,description,location_description,arrest,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,13218359,JG433539,2023-09-21,00:00:00.000,114XX S UNION AVE,0820,THEFT,$500 AND UNDER,STREET,0,...,21,49,06,1173696,1828978,2023,2023-09-28T15:42:39.000,41.686124,-87.639761,POINT(41.686123758 -87.639760794)
1,13220574,JG435752,2023-09-21,00:00:00.000,070XX N Sheridan Rd,0820,THEFT,$500 AND UNDER,STREET,0,...,49,1,06,1166538,1947205,2023,2023-09-28T15:42:39.000,42.010705,-87.662584,POINT(42.010704854 -87.662584365)
2,13222674,JG434826,2023-09-21,00:00:00.000,007XX N LA SALLE DR,0460,BATTERY,SIMPLE,COMMERCIAL / BUSINESS OFFICE,0,...,42,8,08B,1175002,1905476,2023,2023-09-28T15:42:39.000,41.896013,-87.632698,POINT(41.896013242 -87.632697553)
3,13219537,JG435105,2023-09-21,00:00:00.000,034XX W CULLOM AVE,0890,THEFT,FROM BUILDING,RESIDENCE,0,...,33,16,06,1152712,1928368,2023,2023-09-28T15:42:39.000,41.959301,-87.713957,POINT(41.959301089 -87.713956891)
4,13217762,JG432890,2023-09-21,00:00:00.000,0000X S COLUMBUS DR,0920,MOTOR VEHICLE THEFT,ATTEMPT - AUTOMOBILE,STREET,0,...,42,32,07,1178293,1900315,2023,2023-09-28T15:42:39.000,41.881777,-87.620768,POINT(41.881776822 -87.620767948)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50630,13141484,JG342175,2023-07-15,08:00:00.000,039XX W FLOURNOY ST,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,0,...,24,26,05,1150263,1896740,2023,2023-08-19T15:40:26.000,41.872559,-87.723787,POINT(41.872559142 -87.723786945)
50631,13141591,JG342310,2023-07-15,08:00:00.000,067XX S CRANDON AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,0,...,5,43,08A,1192484,1860691,2023,2023-08-19T15:40:26.000,41.772712,-87.569953,POINT(41.772711564 -87.569952861)
50632,13141246,JG341856,2023-07-15,08:00:00.000,071XX S PAXTON AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,0,...,5,43,07,1192124,1857980,2023,2023-08-19T15:40:26.000,41.765281,-87.571361,POINT(41.765281136 -87.57136054)
50633,13145984,JG347492,2023-07-15,08:00:00.000,052XX S KEATING AVE,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER (SPECIFY),0,...,23,56,26,1145578,1869507,2023,2023-08-19T15:40:26.000,41.797918,-87.741677,POINT(41.79791797 -87.741676545)


In [12]:
df.columns = ["id","case_number","date", "time", "block","iucr","primary_type","description","location_desc","arrest","domestic","beat","district","ward", "community_area", "fbi_code", "x_coord",
       "y_coord","year","updated_on","latitude","longitude", "location"]
df.columns  

Index(['id', 'case_number', 'date', 'time', 'block', 'iucr', 'primary_type',
       'description', 'location_desc', 'arrest', 'domestic', 'beat',
       'district', 'ward', 'community_area', 'fbi_code', 'x_coord', 'y_coord',
       'year', 'updated_on', 'latitude', 'longitude', 'location'],
      dtype='object')

#### Now we are going to delete the rows where location value is null:

In [13]:
df= df.dropna(subset=['location'])

df.head()

,id,case_number,date,time,block,iucr,primary_type,description,location_desc,arrest,...,ward,community_area,fbi_code,x_coord,y_coord,year,updated_on,latitude,longitude,location
0,13218359,JG433539,2023-09-21,00:00:00.000,114XX S UNION AVE,0820,THEFT,$500 AND UNDER,STREET,0,...,21,49,06,1173696,1828978,2023,2023-09-28T15:42:39.000,41.686124,-87.639761,POINT(41.686123758 -87.639760794)
1,13220574,JG435752,2023-09-21,00:00:00.000,070XX N Sheridan Rd,0820,THEFT,$500 AND UNDER,STREET,0,...,49,1,06,1166538,1947205,2023,2023-09-28T15:42:39.000,42.010705,-87.662584,POINT(42.010704854 -87.662584365)
2,13222674,JG434826,2023-09-21,00:00:00.000,007XX N LA SALLE DR,0460,BATTERY,SIMPLE,COMMERCIAL / BUSINESS OFFICE,0,...,42,8,08B,1175002,1905476,2023,2023-09-28T15:42:39.000,41.896013,-87.632698,POINT(41.896013242 -87.632697553)
3,13219537,JG435105,2023-09-21,00:00:00.000,034XX W CULLOM AVE,0890,THEFT,FROM BUILDING,RESIDENCE,0,...,33,16,06,1152712,1928368,2023,2023-09-28T15:42:39.000,41.959301,-87.713957,POINT(41.959301089 -87.713956891)
4,13217762,JG432890,2023-09-21,00:00:00.000,0000X S COLUMBUS DR,0920,MOTOR VEHICLE THEFT,ATTEMPT - AUTOMOBILE,STREET,0,...,42,32,07,1178293,1900315,2023,2023-09-28T15:42:39.000,41.881777,-87.620768,POINT(41.881776822 -87.620767948)


#### Now, we delete the columns that are not present in the db table

In [14]:
columns_to_drop = ['case_number', 'domestic', 'beat', 'ward', 'community_area', 'fbi_code', 'x_coord', 'y_coord', 'latitude', 'longitude']
df.drop(columns=columns_to_drop, inplace=True)
df

,id,date,time,block,iucr,primary_type,description,location_desc,arrest,district,year,updated_on,location
0,13218359,2023-09-21,00:00:00.000,114XX S UNION AVE,0820,THEFT,$500 AND UNDER,STREET,0,22,2023,2023-09-28T15:42:39.000,POINT(41.686123758 -87.639760794)
1,13220574,2023-09-21,00:00:00.000,070XX N Sheridan Rd,0820,THEFT,$500 AND UNDER,STREET,0,24,2023,2023-09-28T15:42:39.000,POINT(42.010704854 -87.662584365)
2,13222674,2023-09-21,00:00:00.000,007XX N LA SALLE DR,0460,BATTERY,SIMPLE,COMMERCIAL / BUSINESS OFFICE,0,18,2023,2023-09-28T15:42:39.000,POINT(41.896013242 -87.632697553)
3,13219537,2023-09-21,00:00:00.000,034XX W CULLOM AVE,0890,THEFT,FROM BUILDING,RESIDENCE,0,17,2023,2023-09-28T15:42:39.000,POINT(41.959301089 -87.713956891)
4,13217762,2023-09-21,00:00:00.000,0000X S COLUMBUS DR,0920,MOTOR VEHICLE THEFT,ATTEMPT - AUTOMOBILE,STREET,0,1,2023,2023-09-28T15:42:39.000,POINT(41.881776822 -87.620767948)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50630,13141484,2023-07-15,08:00:00.000,039XX W FLOURNOY ST,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,0,11,2023,2023-08-19T15:40:26.000,POINT(41.872559142 -87.723786945)
50631,13141591,2023-07-15,08:00:00.000,067XX S CRANDON AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,0,3,2023,2023-08-19T15:40:26.000,POINT(41.772711564 -87.569952861)
50632,13141246,2023-07-15,08:00:00.000,071XX S PAXTON AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,0,3,2023,2023-08-19T15:40:26.000,POINT(41.765281136 -87.57136054)
50633,13145984,2023-07-15,08:00:00.000,052XX S KEATING AVE,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER (SPECIFY),0,8,2023,2023-08-19T15:40:26.000,POINT(41.79791797 -87.741676545)


In [15]:
columns_to_drop = ['primary_type', 'description']
df.drop(columns=columns_to_drop, inplace=True)
df

,id,date,time,block,iucr,location_desc,arrest,district,year,updated_on,location
0,13218359,2023-09-21,00:00:00.000,114XX S UNION AVE,0820,STREET,0,22,2023,2023-09-28T15:42:39.000,POINT(41.686123758 -87.639760794)
1,13220574,2023-09-21,00:00:00.000,070XX N Sheridan Rd,0820,STREET,0,24,2023,2023-09-28T15:42:39.000,POINT(42.010704854 -87.662584365)
2,13222674,2023-09-21,00:00:00.000,007XX N LA SALLE DR,0460,COMMERCIAL / BUSINESS OFFICE,0,18,2023,2023-09-28T15:42:39.000,POINT(41.896013242 -87.632697553)
3,13219537,2023-09-21,00:00:00.000,034XX W CULLOM AVE,0890,RESIDENCE,0,17,2023,2023-09-28T15:42:39.000,POINT(41.959301089 -87.713956891)
4,13217762,2023-09-21,00:00:00.000,0000X S COLUMBUS DR,0920,STREET,0,1,2023,2023-09-28T15:42:39.000,POINT(41.881776822 -87.620767948)
...,...,...,...,...,...,...,...,...,...,...,...
50630,13141484,2023-07-15,08:00:00.000,039XX W FLOURNOY ST,0610,RESIDENCE,0,11,2023,2023-08-19T15:40:26.000,POINT(41.872559142 -87.723786945)
50631,13141591,2023-07-15,08:00:00.000,067XX S CRANDON AVE,2820,APARTMENT,0,3,2023,2023-08-19T15:40:26.000,POINT(41.772711564 -87.569952861)
50632,13141246,2023-07-15,08:00:00.000,071XX S PAXTON AVE,0910,STREET,0,3,2023,2023-08-19T15:40:26.000,POINT(41.765281136 -87.57136054)
50633,13145984,2023-07-15,08:00:00.000,052XX S KEATING AVE,2826,OTHER (SPECIFY),0,8,2023,2023-08-19T15:40:26.000,POINT(41.79791797 -87.741676545)


#### This is our data to insert

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50297 entries, 0 to 50634
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             50297 non-null  int64 
 1   date           50297 non-null  object
 2   time           50297 non-null  object
 3   block          50297 non-null  object
 4   iucr           50297 non-null  object
 5   location_desc  50297 non-null  object
 6   arrest         50297 non-null  int32 
 7   district       50297 non-null  int32 
 8   year           50297 non-null  int64 
 9   updated_on     50297 non-null  object
 10  location       50297 non-null  object
dtypes: int32(2), int64(2), object(7)
memory usage: 4.2+ MB


#### INSERT THE DATA TO MySQL TABLE CRIMES

In [17]:
mycursor = conx.cursor()

query2 = """INSERT INTO crimes (id, date, time, block, iucr, location_desc, arrest, district, year, updated_on, location) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, ST_GeomFromText(%s))
ON DUPLICATE KEY UPDATE
date = VALUES(date), time = VALUES(time), block = VALUES(block), iucr = VALUES(iucr), location_desc = VALUES(location_desc),
arrest = VALUES(arrest), district = VALUES(district), year = VALUES(year), updated_on = VALUES(updated_on), location = VALUES(location)
"""

for index, row in df.iterrows():
    datos = (row['id'], row['date'], row['time'], row['block'],
             row['iucr'], row['location_desc'],
             row['arrest'], row['district'], row['year'], row['updated_on'], row['location'])

    mycursor.execute(query2, datos)

conx.commit()
mycursor.close()

True